In [1]:
# IMPORTING PACKAGES
# ------------------------------------------------------
import pandas as pd
import numpy as np
import os
import re
import warnings

# VISUALIZATION
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt
%matplotlib inline
pd.options.display.max_columns = 0
pd.options.display.max_colwidth =10

# CONFIGURATIONS
# ------------------------------------------------------
warnings.filterwarnings("ignore")

# MODEL CONFIG
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [2]:
os.chdir("data")

In [3]:
df = pd.read_csv("CleanedData.csv")

In [4]:
df["Category_name"].value_counts()

tv, television, led                2266
drive, storage, hard               2105
headphone, earbud, bluetooth       1801
tv, 4k, television                 1636
headphone                          1580
receiver, amplifier, home          1541
car, gps, dash                     1464
camera, shoot, point               1386
tablet, computer, ipad             1363
camera, mirrorless                 1302
laptop, computer                   1210
speaker, portable, bluetooth       1202
player, ipod                       1098
sound, speaker, home               1088
car, speaker, subwoofer            1072
speaker, audio                     1035
case, computer                      926
laptop, computer, macbook           925
speaker, bluetooth, mobile          841
phone, cell                         789
mount, tv, stand                    783
camera, camcorder, action           750
wireless, speaker, audio            696
lens, camera                        683
monitor, computer                   654


## Price Elasticity for all Products.

In [5]:
#dfx = df[df["Category_name"].str.contains("speaker",case=False)]
dfx= df[(df["Year"]==2017) | (df["Year"]==2018)]

In [6]:
dfx = dfx.reset_index(drop=True)

In [7]:
dfx.sort_values(by=['Date'],ignore_index=True)

,id,Name,Brand,Description,Category_name,Merchant,Actual_price,Discount_price,isSale,Discount_per,Imp_count,Day,Month,Year,Week,Month_Name,Day_Name,Year_Month,Year_Week,Date
0,AVpfO3...,KS9800...,Samsung,"TV, Vi...","tv, 4k...",Walmar...,3497.99,3497.99,No,0.0,1,5,1,2017,1,January,Thursday,2017-01,"2017,01",2017-0...
1,AVpfiN...,Yamaha...,Yamaha,Electr...,receiv...,Walmar...,473.65,473.65,No,0.0,1,5,1,2017,1,January,Thursday,2017-01,"2017,01",2017-0...
2,AVpgQi...,XDP-10...,Pioneer,TVs En...,player...,Walmar...,399.99,399.99,No,0.0,1,5,1,2017,1,January,Thursday,2017-01,"2017,01",2017-0...
3,AVpgQi...,XDP-10...,Pioneer,TVs En...,player...,Walmar...,599.00,599.00,No,0.0,1,7,1,2017,1,January,Saturday,2017-01,"2017,01",2017-0...
4,AVpfWO...,Alpine...,Alpine,Electr...,"car, g...",Walmar...,137.99,137.99,No,0.0,1,7,1,2017,1,January,Saturday,2017-01,"2017,01",2017-0...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40646,AVpfWu...,Source...,NZXT,Comput...,"case, ...",bhphot...,99.90,99.90,No,0.0,1,25,7,2018,30,July,Wednesday,2018-07,"2018,30",2018-0...
40647,AV1YFi...,1TB WD...,WD,Comput...,"drive,...",bhphot...,69.99,69.99,No,0.0,7,25,7,2018,30,July,Wednesday,2018-07,"2018,30",2018-0...
40648,AV1YFj...,MyWire...,Actiontec,Stream...,"tv, st...",bhphot...,199.99,199.99,No,0.0,1,25,7,2018,30,July,Wednesday,2018-07,"2018,30",2018-0...
40649,AVpfJp...,SL-15 ...,Cerwin...,Floor ...,speake...,bhphot...,449.00,449.00,No,0.0,9,25,7,2018,30,July,Wednesday,2018-07,"2018,30",2018-0...


In [8]:
#Grouping data by Name and Week number.
dfx_mod = dfx.groupby(['Name', "Year_Month"]).agg({'Discount_price':'mean','Imp_count':'sum',"Category_name":lambda x: pd.Series.mode(x)[0],"Date":lambda x: pd.Series.mode(x)[0]}).reset_index()


In [10]:
#Rearranging
dfx_mod = dfx_mod[["Name","Category_name","Discount_price","Imp_count","Year_Month","Date"]]

In [12]:
dfx_mod['Date']= pd.to_datetime(dfx_mod['Date'])


In [23]:

dfx_mod.sort_values("Year_Month",ignore_index=True)

,Name,Category_name,Discount_price,Imp_count,Year_Month,Date
0,Logite...,"case, ...",59.9900,1,2017-01,2017-01-08
1,PMX840...,microp...,223.1700,2,2017-01,2017-01-26
2,Magell...,"car, g...",132.6925,4,2017-01,2017-01-20
3,Alpine...,"car, g...",137.9900,1,2017-01,2017-01-07
4,Dell -...,laptop...,1399.9900,9,2017-01,2017-01-24
...,...,...,...,...,...,...
10839,PowerS...,camera...,699.0000,12,2018-07,2018-07-25
10840,DP-300...,turnta...,329.0000,7,2018-07,2018-07-25
10841,Alpha ...,camera...,780.0000,20,2018-07,2018-07-25
10842,MG278Q...,monito...,549.0000,10,2018-07,2018-07-25


#### X-Values = PRICE

In [22]:

#New X dataframe.
x_new = dfx_mod.pivot(index="Year_Week", columns='Name' ,values='Discount_price')
print("Structure of DataFrame = ",x_new.shape)
x_new.shape

Structure of DataFrame =  (80, 1223)


(80, 1223)

In [24]:
x_new.head(9999)

Name,1080p Day/Night PTZ Speed Dome Camera with 5 to 61mm Auto Focus Lens,10TB G-DRIVE with Thunderbolt,"12 MacBook (Mid 2017, Gold)","12 MacBook (Mid 2017, Silver)",12.3 32GB Multi-Touch 2-in-1 Chromebook Plus,120 Bulb LED Video Light (Daylight),128GB iPod touch (Gold) (6th Generation),128GB iPod touch (Space Gray) (6th Generation),"13.3 MacBook Air (Mid 2017, Silver)",14 Checkpoint Friendly Laptop Case,"15-Volt 3 Amp 6.0 mm-3.0 mm AC Adapter for TOSHIBA Tecra, Satellite and Satellite Pro Series Laptops","15.4 MacBook Pro with Touch Bar (Late 2016, Silver)","15.4 MacBook Pro with Touch Bar (Late 2016, Space Gray)",1500-Watt Digital Infrared Heater with Wi-Fi Phone App - White,1506TSA Lock,151 SE Outdoor Environmental Speakers (White),17.3 WE72 7RJ Mobile Workstation,1TB G-Drive Mobile Hard Drive with Thunderbolt,1TB T5 Portable Solid-State Drive (Black),1TB WD Black Mobile OEM Hard Drive (WD10JPLX),"2) New Kicker 41KSC6934 6x9 3-Way 300 Watt Car Audio Coaxial Speakers KSC693""",21.5 iMac (Mid 2017),21.5 iMac with Retina 4K Display (Mid 2017),22 Widescreen Full HD 1080p LED Monitor,24-70mm f/2.8 GM Lens and 82mm Circular Polarizer Filter Kit,240GB SSD Plus SATA III 2.5 Internal SSD,251 Outdoor Environmental Speakers (White),27 iMac with Retina 5K Display (Mid 2017),2PJ4280 - Peerless-AV Wall Mount for Tablet PC,2TB My Passport USB 3.0 Secure Portable Hard Drive (Orange),2TB Red 5400 rpm SATA III 3.5 Internal NAS HDD,32GB High Speed UHS-I SDHC U3 Memory Card (Class 10),32GB iPod touch (Gold) (6th Generation),32GB iPod touch (Pink) (6th Generation),391 Series C32F391 32 16:9 Curved FreeSync LCD Monitor,3TB AirPort Time Capsule (5th Generation),450 Mbps 2.4/5GHz Wireless N Dual Band PCI Express Adapter,480GB Extreme Pro Solid State Drive,4GB NW-WS413 Sports Walkman Digital Music Player (Black),4K Video Camera,...,Yamaha - WX-010 Wireless Speaker for Streaming Music - Black,Yamaha R-N301 Network Audio/Video Player - Black (rn301bl),Yamaha RX-V379BL 5.1-Channel AV Receiver (Black),Yamaha RX-V383BL 5.1-Channel 4K Ultra HD AV Receiver with Bluetooth,Yamaha RX-V479BL 5.1-Channel AV Receiver (Black),Yamaha RX-V681BL 7.2-Channel MusicCast AV Receiver with Bluetooth,Yamaha WX-030BL MusicCast Wireless Speaker with Wi-Fi and Bluetooth (Black),ZS-RS60BT CD Boombox,ZTE - Spro 2 Wireless Smart DLP Projector - Silver,ZTE Warp Elite No Contract Phone - Retail Packaging - Boost,Zivix - puc+ Bluetooth MIDI Interface - Black,Zoom - H1 Handy Recorder - Black,Zubie - In-Car Wi-Fi and Vehicle Monitoring Device - Black,e1659Fwu 16 Widescreen USB 3.1 Gen 1 Powered LED Backlit LCD Monitor,fr Case for Galaxy S7 (Black),h.ear go Wireless Speaker (Bordeaux Pink),h.ear go Wireless Speaker (Cinnabar Red),h.ear go Wireless Speaker (Lime Yellow),h.ear go Wireless Speaker (Viridian Blue),i.am+ BUTTONS,iHome Rechargeable Splash Proof Stereo Bluetooth Speaker - Black (IBT33BC),iHome iBN43BC Bluetooth Stereo Dual Alarm FM Clock Radio and Speakerphone with USB Charging,iLive - ISBW216 Wireless Gnome Outdoor Speaker - Multi,iPL24 FM Clock Radio with Lightning Dock and USB Port (Gunmetal),iPhone 7 Smart Battery Case (Black),iSimple - BluStream Bluetooth Factory Radio Module - Black,"iSimple - StrongHold Headrest Mount for Most 7 - 10.2"" Tablets - Black""",iSimple - TranzIt Bluetooth Factory Radio Module - Black,iSimple ISBC01 BluClik Bluetooth Remote Control with Steering Wheel and Dash Mounts,inactive,instax mini 8 Instant Film Camera and Groovy Case Kit (Raspberry),instax mini Rainbow Instant Film (10 Exposures),j5create - VGA to HDMI Video adapter - White,"kate spade new york - Glitter Sleeve for 13 Apple® MacBook® - Rose Gold""",kenwood pkg-mr362bt marine cd receiver with bluetooth and 6.5 2 way speakers package,mophie - Juice Pack External Battery Case with Wireless Charging for Apple® iPhone® 7 Plus and 8 Plus - Rose Gold,mophie powerstation Plus Mini External Battery with Built in Cables for Smartphones and Tablets (4,myCharge - RAZOR PLUS USB Portable Power 

In [15]:
# FOR X VALUES
#Filling missing values with fillna + forward filling
x_new_ = pd.DataFrame(x_new.to_records())
x_new_.fillna(method='ffill', inplace=True)
x_new_.fillna(method='bfill', inplace=True)
x_new_.head(5)

,Week_Year,1080p Day/Night PTZ Speed Dome Camera with 5 to 61mm Auto Focus Lens,10TB G-DRIVE with Thunderbolt,"12 MacBook (Mid 2017, Gold)","12 MacBook (Mid 2017, Silver)",12.3 32GB Multi-Touch 2-in-1 Chromebook Plus,120 Bulb LED Video Light (Daylight),128GB iPod touch (Gold) (6th Generation),128GB iPod touch (Space Gray) (6th Generation),"13.3 MacBook Air (Mid 2017, Silver)",14 Checkpoint Friendly Laptop Case,"15-Volt 3 Amp 6.0 mm-3.0 mm AC Adapter for TOSHIBA Tecra, Satellite and Satellite Pro Series Laptops","15.4 MacBook Pro with Touch Bar (Late 2016, Silver)","15.4 MacBook Pro with Touch Bar (Late 2016, Space Gray)",1500-Watt Digital Infrared Heater with Wi-Fi Phone App - White,1506TSA Lock,151 SE Outdoor Environmental Speakers (White),17.3 WE72 7RJ Mobile Workstation,1TB G-Drive Mobile Hard Drive with Thunderbolt,1TB T5 Portable Solid-State Drive (Black),1TB WD Black Mobile OEM Hard Drive (WD10JPLX),"2) New Kicker 41KSC6934 6x9 3-Way 300 Watt Car Audio Coaxial Speakers KSC693""",21.5 iMac (Mid 2017),21.5 iMac with Retina 4K Display (Mid 2017),22 Widescreen Full HD 1080p LED Monitor,24-70mm f/2.8 GM Lens and 82mm Circular Polarizer Filter Kit,240GB SSD Plus SATA III 2.5 Internal SSD,251 Outdoor Environmental Speakers (White),27 iMac with Retina 5K Display (Mid 2017),2PJ4280 - Peerless-AV Wall Mount for Tablet PC,2TB My Passport USB 3.0 Secure Portable Hard Drive (Orange),2TB Red 5400 rpm SATA III 3.5 Internal NAS HDD,32GB High Speed UHS-I SDHC U3 Memory Card (Class 10),32GB iPod touch (Gold) (6th Generation),32GB iPod touch (Pink) (6th Generation),391 Series C32F391 32 16:9 Curved FreeSync LCD Monitor,3TB AirPort Time Capsule (5th Generation),450 Mbps 2.4/5GHz Wireless N Dual Band PCI Express Adapter,480GB Extreme Pro Solid State Drive,4GB NW-WS413 Sports Walkman Digital Music Player (Black),...,Yamaha - WX-010 Wireless Speaker for Streaming Music - Black,Yamaha R-N301 Network Audio/Video Player - Black (rn301bl),Yamaha RX-V379BL 5.1-Channel AV Receiver (Black),Yamaha RX-V383BL 5.1-Channel 4K Ultra HD AV Receiver with Bluetooth,Yamaha RX-V479BL 5.1-Channel AV Receiver (Black),Yamaha RX-V681BL 7.2-Channel MusicCast AV Receiver with Bluetooth,Yamaha WX-030BL MusicCast Wireless Speaker with Wi-Fi and Bluetooth (Black),ZS-RS60BT CD Boombox,ZTE - Spro 2 Wireless Smart DLP Projector - Silver,ZTE Warp Elite No Contract Phone - Retail Packaging - Boost,Zivix - puc+ Bluetooth MIDI Interface - Black,Zoom - H1 Handy Recorder - Black,Zubie - In-Car Wi-Fi and Vehicle Monitoring Device - Black,e1659Fwu 16 Widescreen USB 3.1 Gen 1 Powered LED Backlit LCD Monitor,fr Case for Galaxy S7 (Black),h.ear go Wireless Speaker (Bordeaux Pink),h.ear go Wireless Speaker (Cinnabar Red),h.ear go Wireless Speaker (Lime Yellow),h.ear go Wireless Speaker (Viridian Blue),i.am+ BUTTONS,iHome Rechargeable Splash Proof Stereo Bluetooth Speaker - Black (IBT33BC),iHome iBN43BC Bluetooth Stereo Dual Alarm FM Clock Radio and Speakerphone with USB Charging,iLive - ISBW216 Wireless Gnome Outdoor Speaker - Multi,iPL24 FM Clock Radio with Lightning Dock and USB Port (Gunmetal),iPhone 7 Smart Battery Case (Black),iSimple - BluStream Bluetooth Factory Radio Module - Black,"iSimple - StrongHold Headrest Mount for Most 7 - 10.2"" Tablets - Black""",iSimple - TranzIt Bluetooth Factory Radio Module - Black,iSimple ISBC01 BluClik Bluetooth Remote Control with Steering Wheel and Dash Mounts,inactive,instax mini 8 Instant Film Camera and Groovy Case Kit (Raspberry),instax mini Rainbow Instant Film (10 Exposures),j5create - VGA to HDMI Video adapter - White,"kate spade new york - Glitter Sleeve for 13 Apple® MacBook® - Rose Gold""",kenwood pkg-mr362bt marine cd receiver with bluetooth and 6.5 2 way speakers package,mophie - Juice Pack External Battery Case with Wireless Charging for Apple® iPhone® 7 Plus and 8 Plus - Rose Gold,mophie powerstation Plus Mini External Battery with Built in Cables for Smartphones and Tablets (4,myCharge - RAZOR PLUS USB Portable Power Bank - Sil

#### Y-Values = DEMAND or IMPRESSIONS

In [13]:
#New Y dataframe.
y_new = dfx_mod.pivot(index='Week', columns='Name', values='Imp_count')
print("Structure of DataFrame = ",y_new.shape)
y_new.sample(5)

Structure of DataFrame =  (52, 1223)


Name,1080p Day/Night PTZ Speed Dome Camera with 5 to 61mm Auto Focus Lens,10TB G-DRIVE with Thunderbolt,"12 MacBook (Mid 2017, Gold)","12 MacBook (Mid 2017, Silver)",12.3 32GB Multi-Touch 2-in-1 Chromebook Plus,120 Bulb LED Video Light (Daylight),128GB iPod touch (Gold) (6th Generation),128GB iPod touch (Space Gray) (6th Generation),"13.3 MacBook Air (Mid 2017, Silver)",14 Checkpoint Friendly Laptop Case,"15-Volt 3 Amp 6.0 mm-3.0 mm AC Adapter for TOSHIBA Tecra, Satellite and Satellite Pro Series Laptops","15.4 MacBook Pro with Touch Bar (Late 2016, Silver)","15.4 MacBook Pro with Touch Bar (Late 2016, Space Gray)",1500-Watt Digital Infrared Heater with Wi-Fi Phone App - White,1506TSA Lock,151 SE Outdoor Environmental Speakers (White),17.3 WE72 7RJ Mobile Workstation,1TB G-Drive Mobile Hard Drive with Thunderbolt,1TB T5 Portable Solid-State Drive (Black),1TB WD Black Mobile OEM Hard Drive (WD10JPLX),"2) New Kicker 41KSC6934 6x9 3-Way 300 Watt Car Audio Coaxial Speakers KSC693""",21.5 iMac (Mid 2017),21.5 iMac with Retina 4K Display (Mid 2017),22 Widescreen Full HD 1080p LED Monitor,24-70mm f/2.8 GM Lens and 82mm Circular Polarizer Filter Kit,240GB SSD Plus SATA III 2.5 Internal SSD,251 Outdoor Environmental Speakers (White),27 iMac with Retina 5K Display (Mid 2017),2PJ4280 - Peerless-AV Wall Mount for Tablet PC,2TB My Passport USB 3.0 Secure Portable Hard Drive (Orange),2TB Red 5400 rpm SATA III 3.5 Internal NAS HDD,32GB High Speed UHS-I SDHC U3 Memory Card (Class 10),32GB iPod touch (Gold) (6th Generation),32GB iPod touch (Pink) (6th Generation),391 Series C32F391 32 16:9 Curved FreeSync LCD Monitor,3TB AirPort Time Capsule (5th Generation),450 Mbps 2.4/5GHz Wireless N Dual Band PCI Express Adapter,480GB Extreme Pro Solid State Drive,4GB NW-WS413 Sports Walkman Digital Music Player (Black),4K Video Camera,...,Yamaha - WX-010 Wireless Speaker for Streaming Music - Black,Yamaha R-N301 Network Audio/Video Player - Black (rn301bl),Yamaha RX-V379BL 5.1-Channel AV Receiver (Black),Yamaha RX-V383BL 5.1-Channel 4K Ultra HD AV Receiver with Bluetooth,Yamaha RX-V479BL 5.1-Channel AV Receiver (Black),Yamaha RX-V681BL 7.2-Channel MusicCast AV Receiver with Bluetooth,Yamaha WX-030BL MusicCast Wireless Speaker with Wi-Fi and Bluetooth (Black),ZS-RS60BT CD Boombox,ZTE - Spro 2 Wireless Smart DLP Projector - Silver,ZTE Warp Elite No Contract Phone - Retail Packaging - Boost,Zivix - puc+ Bluetooth MIDI Interface - Black,Zoom - H1 Handy Recorder - Black,Zubie - In-Car Wi-Fi and Vehicle Monitoring Device - Black,e1659Fwu 16 Widescreen USB 3.1 Gen 1 Powered LED Backlit LCD Monitor,fr Case for Galaxy S7 (Black),h.ear go Wireless Speaker (Bordeaux Pink),h.ear go Wireless Speaker (Cinnabar Red),h.ear go Wireless Speaker (Lime Yellow),h.ear go Wireless Speaker (Viridian Blue),i.am+ BUTTONS,iHome Rechargeable Splash Proof Stereo Bluetooth Speaker - Black (IBT33BC),iHome iBN43BC Bluetooth Stereo Dual Alarm FM Clock Radio and Speakerphone with USB Charging,iLive - ISBW216 Wireless Gnome Outdoor Speaker - Multi,iPL24 FM Clock Radio with Lightning Dock and USB Port (Gunmetal),iPhone 7 Smart Battery Case (Black),iSimple - BluStream Bluetooth Factory Radio Module - Black,"iSimple - StrongHold Headrest Mount for Most 7 - 10.2"" Tablets - Black""",iSimple - TranzIt Bluetooth Factory Radio Module - Black,iSimple ISBC01 BluClik Bluetooth Remote Control with Steering Wheel and Dash Mounts,inactive,instax mini 8 Instant Film Camera and Groovy Case Kit (Raspberry),instax mini Rainbow Instant Film (10 Exposures),j5create - VGA to HDMI Video adapter - White,"kate spade new york - Glitter Sleeve for 13 Apple® MacBook® - Rose Gold""",kenwood pkg-mr362bt marine cd receiver with bluetooth and 6.5 2 way speakers package,mophie - Juice Pack External Battery Case with Wireless Charging for Apple® iPhone® 7 Plus and 8 Plus - Rose Gold,mophie powerstation Plus Mini External Battery with Built in Cables for Smartphones and Tablets (4,myCharge - RAZOR PLUS USB Portable Power 

In [14]:
# FOR Y VALUES
#Filling missing values with fillna + forward filling
y_new_ = pd.DataFrame(y_new.to_records())
y_new_.fillna(method='ffill', inplace=True)
y_new_.fillna(method='bfill', inplace=True)
y_new_.head(5)

,Week,1080p Day/Night PTZ Speed Dome Camera with 5 to 61mm Auto Focus Lens,10TB G-DRIVE with Thunderbolt,"12 MacBook (Mid 2017, Gold)","12 MacBook (Mid 2017, Silver)",12.3 32GB Multi-Touch 2-in-1 Chromebook Plus,120 Bulb LED Video Light (Daylight),128GB iPod touch (Gold) (6th Generation),128GB iPod touch (Space Gray) (6th Generation),"13.3 MacBook Air (Mid 2017, Silver)",14 Checkpoint Friendly Laptop Case,"15-Volt 3 Amp 6.0 mm-3.0 mm AC Adapter for TOSHIBA Tecra, Satellite and Satellite Pro Series Laptops","15.4 MacBook Pro with Touch Bar (Late 2016, Silver)","15.4 MacBook Pro with Touch Bar (Late 2016, Space Gray)",1500-Watt Digital Infrared Heater with Wi-Fi Phone App - White,1506TSA Lock,151 SE Outdoor Environmental Speakers (White),17.3 WE72 7RJ Mobile Workstation,1TB G-Drive Mobile Hard Drive with Thunderbolt,1TB T5 Portable Solid-State Drive (Black),1TB WD Black Mobile OEM Hard Drive (WD10JPLX),"2) New Kicker 41KSC6934 6x9 3-Way 300 Watt Car Audio Coaxial Speakers KSC693""",21.5 iMac (Mid 2017),21.5 iMac with Retina 4K Display (Mid 2017),22 Widescreen Full HD 1080p LED Monitor,24-70mm f/2.8 GM Lens and 82mm Circular Polarizer Filter Kit,240GB SSD Plus SATA III 2.5 Internal SSD,251 Outdoor Environmental Speakers (White),27 iMac with Retina 5K Display (Mid 2017),2PJ4280 - Peerless-AV Wall Mount for Tablet PC,2TB My Passport USB 3.0 Secure Portable Hard Drive (Orange),2TB Red 5400 rpm SATA III 3.5 Internal NAS HDD,32GB High Speed UHS-I SDHC U3 Memory Card (Class 10),32GB iPod touch (Gold) (6th Generation),32GB iPod touch (Pink) (6th Generation),391 Series C32F391 32 16:9 Curved FreeSync LCD Monitor,3TB AirPort Time Capsule (5th Generation),450 Mbps 2.4/5GHz Wireless N Dual Band PCI Express Adapter,480GB Extreme Pro Solid State Drive,4GB NW-WS413 Sports Walkman Digital Music Player (Black),...,Yamaha - WX-010 Wireless Speaker for Streaming Music - Black,Yamaha R-N301 Network Audio/Video Player - Black (rn301bl),Yamaha RX-V379BL 5.1-Channel AV Receiver (Black),Yamaha RX-V383BL 5.1-Channel 4K Ultra HD AV Receiver with Bluetooth,Yamaha RX-V479BL 5.1-Channel AV Receiver (Black),Yamaha RX-V681BL 7.2-Channel MusicCast AV Receiver with Bluetooth,Yamaha WX-030BL MusicCast Wireless Speaker with Wi-Fi and Bluetooth (Black),ZS-RS60BT CD Boombox,ZTE - Spro 2 Wireless Smart DLP Projector - Silver,ZTE Warp Elite No Contract Phone - Retail Packaging - Boost,Zivix - puc+ Bluetooth MIDI Interface - Black,Zoom - H1 Handy Recorder - Black,Zubie - In-Car Wi-Fi and Vehicle Monitoring Device - Black,e1659Fwu 16 Widescreen USB 3.1 Gen 1 Powered LED Backlit LCD Monitor,fr Case for Galaxy S7 (Black),h.ear go Wireless Speaker (Bordeaux Pink),h.ear go Wireless Speaker (Cinnabar Red),h.ear go Wireless Speaker (Lime Yellow),h.ear go Wireless Speaker (Viridian Blue),i.am+ BUTTONS,iHome Rechargeable Splash Proof Stereo Bluetooth Speaker - Black (IBT33BC),iHome iBN43BC Bluetooth Stereo Dual Alarm FM Clock Radio and Speakerphone with USB Charging,iLive - ISBW216 Wireless Gnome Outdoor Speaker - Multi,iPL24 FM Clock Radio with Lightning Dock and USB Port (Gunmetal),iPhone 7 Smart Battery Case (Black),iSimple - BluStream Bluetooth Factory Radio Module - Black,"iSimple - StrongHold Headrest Mount for Most 7 - 10.2"" Tablets - Black""",iSimple - TranzIt Bluetooth Factory Radio Module - Black,iSimple ISBC01 BluClik Bluetooth Remote Control with Steering Wheel and Dash Mounts,inactive,instax mini 8 Instant Film Camera and Groovy Case Kit (Raspberry),instax mini Rainbow Instant Film (10 Exposures),j5create - VGA to HDMI Video adapter - White,"kate spade new york - Glitter Sleeve for 13 Apple® MacBook® - Rose Gold""",kenwood pkg-mr362bt marine cd receiver with bluetooth and 6.5 2 way speakers package,mophie - Juice Pack External Battery Case with Wireless Charging for Apple® iPhone® 7 Plus and 8 Plus - Rose Gold,mophie powerstation Plus Mini External Battery with Built in Cables for Smartphones and Tablets (4,myCharge - RAZOR PLUS USB Portable Power Bank - Silver,n

In [15]:
x_new_["BOOM 2 Wireless Bluetooth Speaker - Indigo"].value_counts()

199.99...    37
149.99...     9
162.99...     2
145.99...     2
129.99...     1
147.02...     1
Name: BOOM 2 Wireless Bluetooth Speaker - Indigo, dtype: int64

Price elasticity = Change in Quanity sold / Change in the Price

In [19]:
def multizen(cox):
    temp_df = pd.DataFrame()
    temp_df['x'] = x_new_[cox]
    temp_df['y'] = y_new_[cox]
    x_value = temp_df[['x']]
    y_value = temp_df['y']
    #Creating Linear Model
    model = LinearRegression()
    model.fit(x_value, y_value)

    #Eval
    slope = model.coef_ 
    intercept = model.intercept_

    #####
    #y_pred = model.predict(x_value)
    #r_squared = r2_score(y_value, y_pred)

    #Price elasticity
    

    return cox, slope, intercept


In [21]:
for col in x_new_.columns[1:]:
    multizen(col)
    

##########
Slope:  [-0.28621859]
Intercept:  145.55551800161703
R-squared:  56.50794678295459 %
##########
##########
Slope:  [0.04134264]
Intercept:  -7.596379101960917
R-squared:  13.88184718721761 %
##########
##########
Slope:  [0.08925204]
Intercept:  -116.02152547874198
R-squared:  11.043309268128143 %
##########
##########
Slope:  [0.00744481]
Intercept:  -1.1067525194347159
R-squared:  0.3543191274955171 %
##########
##########
Slope:  [-0.01643771]
Intercept:  13.808650871689874
R-squared:  1.3353550370386102 %
##########
##########
Slope:  [0.10786472]
Intercept:  -4.264418330226492
R-squared:  51.80042208177604 %
##########
##########
Slope:  [0.07700298]
Intercept:  -16.251768286084562
R-squared:  13.363668583514144 %
##########
##########
Slope:  [0.13145466]
Intercept:  -31.754889773542452
R-squared:  30.20607384402997 %
##########
##########
Slope:  [0.12372177]
Intercept:  -98.08559595999667
R-squared:  10.031621765960097 %
##########
##########
Slope:  [0.43889212]
Int

#### Forward Selection Method

In [17]:
from sklearn.datasets import load_iris
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import LogisticRegression

iris = load_iris()
X = iris.data
y = iris.target

lr = LogisticRegression()
sfs = SequentialFeatureSelector(lr, n_features_to_select=3,direction="forward")

sfs.fit(X, y)

sfs.get_support()
sfs.transform(X).shape
print('Selected features:', sfs.get_support(indices=True))

Selected features: [0 2 3]


#### Backward Selection Method

In [18]:
from sklearn.datasets import load_iris
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import LogisticRegression

iris = load_iris()
X = iris.data
y = iris.target

lr = LogisticRegression()
sfs = SequentialFeatureSelector(lr, n_features_to_select=2,direction="backward")

sfs.fit(X, y)

sfs.get_support()
sfs.transform(X).shape
print('Selected features:', sfs.get_support(indices=True))

Selected features: [2 3]


#### Based on P Value

In [19]:
from sklearn.datasets import load_iris
from sklearn.feature_selection import f_regression

# load the Iris dataset
iris = load_iris()

# extract the features and target variable
X = iris.data
y = iris.target

# calculate the F-test statistic and p-value for each feature
f_test, p_values = f_regression(X, y)

# print the p-value of each feature
for i, p_value in enumerate(p_values):
    print(f"Feature {i+1}: p-value = {p_value:.4f}")


# if p < 0.05

Feature 1: p-value = 0.0000
Feature 2: p-value = 0.0000
Feature 3: p-value = 0.0000
Feature 4: p-value = 0.0000
